# Introduction

# Imports

In [1]:
from pathlib import Path
from harbor.plotting.ligands import plot_aligned_ligands

In order to avoiding having this repo depend directly on the asapdiscovery repo, I'm going to comment this out, but we'll use a test example from the covid moonshot molecules:
```
from asapdiscovery.data.testing.test_resources import fetch_test_file
mypath = fetch_test_file("Mpro_combined_labeled.sdf")
``` 

In [2]:
mypath = Path("../data/Mpro_combined_labeled.sdf")

I'm copying this code from the asapdiscovery repo.
Once that is conda installable, I'll make that a dep of this repo and use those tools for loading molecules

# Load Molecules

In [3]:
from rdkit import Chem
mols = Chem.SDMolSupplier(str(mypath))

In [4]:
mols = [mol for mol in mols]

In [5]:
import mols2grid

In [6]:
# define the grid to show the scaffolds
grid = mols2grid.display(mols)

In [7]:
grid

# MCSS-based Clustering

In [8]:
from harbor.clustering.hierarchical import ClusterResults, ClusterCenter, HeirarchicalClustering
from openeye import oechem

In [9]:
mol: Chem.Mol = mols[0]
mol.GetPropsAsDict()

{'SMILES': 'ClC=1C=CC=2NCCC(C(=O)NC=3C=NC=C4C=CC=CC34)C2C1',
 'Dataset': 'Mpro-x12171_0A',
 'Compound_ID': 'ALP-POS-477dc5b7-2'}

In [10]:
oemols = []
mol_ids = []
for rdkit_mol in mols:
    smiles = Chem.MolToSmiles(rdkit_mol)
    properties = rdkit_mol.GetPropsAsDict()
    mol_ids.append(properties["Compound_ID"])
    mol = oechem.OEMol()
    oechem.OESmilesToMol(mol, smiles)
    oemols.append(mol)

In [11]:
from harbor.clustering import hierarchical as h
from importlib import reload
reload(h)

<module 'harbor.clustering.hierarchical' from '/home/feanor/harbor/harbor/clustering/hierarchical.py'>

In [12]:
clusterer = h.HeirarchicalClustering(molecules=oemols, mol_ids=mol_ids)

In [13]:
clusters = clusterer.cluster(max_iterations=50, cutoff=12)

100%|██████████| 576/576 [24:55<00:00,  2.60s/it]


['0_ALP-POS-477dc5b7-2', '0_MAK-UNK-6435e6c2-8', '0_DAN-LON-a5fc619e-3', '0_EDJ-MED-76744c27-4', '0_MED-COV-4280ac29-31', '0_JOR-UNI-2fc98d0b-6', '0_AAR-POS-0daf6b7e-36', '0_NAU-LAT-8502cac5-6', '0_JAG-UCB-52b62a6f-11', '0_DUN-NEW-f8ce3686-23', '0_EDJ-MED-015fb6b4-2', '0_MAT-POS-fa06b69f-6', '0_LON-WEI-b8d98729-18', '0_BEN-DND-f2e727cd-5', '0_MAT-POS-c20a539d-4', '0_EDJ-MED-49816e9b-1', '0_PET-UNK-e44ffd04-1', '0_MAT-POS-fce787c2-6', '0_BEN-DND-93268d01-7', '0_DUN-NEW-f8ce3686-24', '0_VLA-UCB-50c39ae8-7', '0_EDJ-MED-6af13d92-1', '0_MAT-POS-916a2c5a-2', '0_AAR-POS-d2a4d1df-4', '0_JAN-GHE-83b26c96-9', '0_MAT-POS-bb423b95-2', '0_MAT-POS-968e8d9c-1', '0_TRY-UNI-714a760b-18', '0_AAR-POS-0daf6b7e-2', '0_AAR-POS-0daf6b7e-29', '0_TRY-UNI-2eddb1ff-2', '0_EDJ-MED-d08626de-3', '0_LON-WEI-9739a092-9 ', '0_JOR-UNI-2fc98d0b-12', '0_MAT-POS-90fd5f68-2', '0_MAT-POS-932d1078-3', '0_ALP-POS-477dc5b7-5', '0_AAR-POS-0daf6b7e-43', '0_DAR-DIA-23aa0b97-13', '0_RAL-THA-05e671eb-10', '0_MAT-POS-3ccb8ef6-1', '0

New clusters
['0_ALP-POS-477dc5b7-2', '0_ALP-UNI-8d415491-6', '0_DAN-LON-a5fc619e-3', '0_DAN-LON-a5fc619e-8', '0_EDJ-MED-76744c27-4', '0_EDJ-MED-968bafd9-1', '0_MED-COV-4280ac29-31', '0_DUN-NEW-f8ce3686-24', '0_JOR-UNI-2fc98d0b-6', '0_JOR-UNI-2fc98d0b-12', '0_JAG-UCB-52b62a6f-11', '0_LON-WEI-0a73fcb8-7', '0_EDJ-MED-015fb6b4-2', '0_MAT-POS-2e8b2191-11', '0_LON-WEI-b8d98729-18', '0_LON-WEI-b8d98729-8', '0_MAT-POS-c20a539d-4', '0_MAT-POS-90fd5f68-38', '0_EDJ-MED-49816e9b-1', '0_MAT-POS-6344a35d-1', '0_MAT-POS-fce787c2-6', '0_MAT-POS-fce787c2-3', '0_BEN-DND-93268d01-7', '0_ALP-POS-f13221e1-4', '0_VLA-UCB-50c39ae8-7', '0_MAT-POS-090737b9-1', '0_EDJ-MED-6af13d92-1', '0_MAT-POS-916a2c5a-2', '0_AAR-POS-d2a4d1df-4', '0_DUN-NEW-f8ce3686-14', '0_JAN-GHE-83b26c96-9', '0_TRY-UNI-714a760b-18', '0_MAT-POS-bb423b95-2', '0_VLA-UNK-82501c2c-1', '0_MAT-POS-968e8d9c-1', '0_JAG-UCB-119787ef-1', '0_AAR-POS-0daf6b7e-2', '0_LON-WEI-8f408cad-5', '0_TRY-UNI-2eddb1ff-2', '0_TRY-UNI-2eddb1ff-3', '0_EDJ-MED-d08626

100%|██████████| 412/412 [08:50<00:00,  1.29s/it]


['1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7', '1_8', '1_9', '1_10', '1_11', '1_12', '1_13', '1_14', '1_15', '1_16', '1_17', '1_18', '1_19', '1_20', '1_21', '1_22', '1_23', '1_24', '1_25', '1_26', '1_27', '1_28', '1_29', '1_30', '1_31', '1_32', '1_33', '1_34', '1_35', '1_36', '1_37', '1_38', '1_39', '1_40', '1_41', '1_42', '1_43', '1_44', '1_45', '1_46', '1_47', '1_48', '1_49', '1_50', '1_51', '1_52', '1_53', '1_54', '1_55', '1_56', '1_57', '1_58', '1_59', '1_60', '1_61', '1_62', '1_63', '1_64', '1_65', '1_66', '1_67', '1_68', '1_69', '1_70', '1_71', '1_72', '1_73', '1_74', '1_75', '1_76', '1_77', '1_78', '1_79', '1_80', '1_81', '1_82', '1_83', '1_84', '1_85', '1_86', '1_87', '1_88', '1_89', '1_90', '1_91', '1_92', '1_93', '1_94', '1_95', '1_96', '1_97', '1_98', '1_99', '1_100', '1_101', '1_102', '1_103', '1_104', '1_105', '1_106', '1_107', '1_108', '1_109', '1_110', '1_111', '1_112', '1_113', '1_114', '1_115', '1_116', '1_117', '1_118', '1_119', '1_120', '1_121', '1_122', '1

...

['17_0', '17_1', '17_2', '17_3', '14_4', '13_6', '10_10', '6_1', '5_29', '1_90', '0_DAR-DIA-842b4336-13']
[[ 0  7 12  5  6  7  7 11  8 10  8]
 [ 7  0  7 12 12 11  9  4 14  4 14]
 [12  7  0  9  6  5  5 11  9 11  9]
 [ 5 12  9  0 12 10  9  5 15  5 14]
 [ 6 12  6 12  0 12  9  6 13  7 12]
 [ 7 11  6 10 12  0 10  6 12  6 12]
 [ 7  9  6  9  9 10  0  6 11  6 10]
 [11  4 11  5  6  6  6  0  5 12  5]
 [ 8 14  9 15 13 12 11  5  0  5 14]
 [10  3 11  3  7  3  3 12  3  0  3]
 [ 8 14  9 14 12 12 10  5 14  5  0]]
[ 2 10  0  8  8 10  8  9  3  7  8]
[12 14 12 15 13 12 11 12 15 12 14]
0
Pairs [(0, 2)]
Singles []
Outliers []
Ignore [0, 2]
1
1 10 [1 5]
Pairs [(0, 2), (1, 10)]
Singles []
Outliers []
Ignore [0, 2, 1, 10]
2
2
3
3 8 [ 3  4  6 10]
Pairs [(0, 2), (1, 10), (3, 8)]
Singles []
Outliers []
Ignore [0, 2, 1, 10, 3, 8]
4
4 8 [ 3  4  6 10]
Pairs [(0, 2), (1, 10), (3, 8)]
Singles [4]
Outliers []
Ignore [0, 2, 1, 10, 3, 8]
5
5 10 [1 5]
Pairs [(0, 2), (1, 10), (3, 8)]
Singles [4, 5]
Outliers []
Ignore [0, 

100%|██████████| 6/6 [00:00<00:00, 495.90it/s]


['18_0', '18_1', '18_2', '18_3', '14_4', '13_6']
[[ 0  5  5 10  6  5]
 [ 5  0 12  4 12 11]
 [ 5 12  0  5 12 10]
 [10  3  5  0  6  6]
 [ 6 12 12  6  0 12]
 [ 6 11 10  6 12  0]]
[3 4 4 0 5 4]
[10 12 12 10 12 12]
0
1
1 4 [1 2 5]
Pairs [(1, 4)]
Singles []
Outliers [0]
Ignore [1, 4]
2
2 4 [1 2 5]
Pairs [(1, 4)]
Singles [2]
Outliers [0]
Ignore [1, 4]
3
4
4
5
5 4 [1 2 5]
Pairs [(1, 4)]
Singles [2, 5]
Outliers [0, 3]
Ignore [1, 4]
Pairs [(1, 4)]
Singles [2, 5]
Outliers [0, 3]
Ignore [1, 4]
New clusters
['18_1', '14_4']
Singles
['17_3', '5_29', '4_24', '0_VLA-UCB-00f2c2b3-7']
Outliers
['17_0', '17_2', '6_1', '1_90']


100%|██████████| 3/3 [00:00<00:00, 1505.85it/s]

['19_0', '18_2', '13_6']
[[ 0 11 10]
 [11  0 10]
 [10 10  0]]
[1 0 1]
[11 11 10]
0
1
2
Pairs []
Singles []
Outliers [0, 1, 2]
Ignore []
New clusters
[]
Singles
[]
Outliers
['18_1', '14_4', '17_3', '5_29', '4_24', '0_VLA-UCB-00f2c2b3-7']


In [14]:
len(clusters)

102

In [15]:
def get_descendents(cluster):
    descendents = []
    for child in cluster.children:
        if isinstance(child, str):
            descendents.append(cluster)
        else:
            descendents.extend(get_descendents(child))
    return descendents

In [20]:
from harbor.plotting import ligands as l
reload(l)

<module 'harbor.plotting.ligands' from '/home/feanor/harbor/harbor/plotting/ligands.py'>

In [24]:
clusters.keys()

dict_keys(['0_MAK-UNK-6435e6c2-8', '0_AAR-POS-0daf6b7e-36', '0_AAR-POS-0daf6b7e-28', '0_AAR-POS-d2a4d1df-9', '0_AAR-POS-0daf6b7e-5', '0_AAR-POS-f650c5f2-2', '0_VIR-GIT-7b3d3065-2', '0_AAR-POS-d2a4d1df-8', '0_AAR-POS-0daf6b7e-39', '0_MAK-UNK-6435e6c2-7', '0_WAR-XCH-72a8c209-5', '0_AAR-POS-0daf6b7e-37', '0_AAR-POS-d2a4d1df-18', '0_AAR-POS-d2a4d1df-12', '0_AAR-POS-0daf6b7e-40', '0_MAT-POS-7dfc56d9-1', '0_AAR-POS-0daf6b7e-45', '0_NAU-LAT-445f63e5-6', '0_AAR-POS-d2a4d1df-5', '0_AAR-POS-0daf6b7e-20', '0_AAR-POS-0daf6b7e-22', '0_AAR-POS-0daf6b7e-35', '0_AAR-RCN-748c104b-1', '0_ALP-POS-c59291d4-5', '0_TAT-ENA-80bfd3e5-7', '0_AAR-POS-0daf6b7e-24', '0_AAR-POS-d2a4d1df-15', '0_TOB-UNK-c2aba166-1', '0_AAR-POS-d2a4d1df-17', '0_AAR-POS-f650c5f2-3', '0_JAG-UCB-cedd89ab-1', '0_AAR-POS-d2a4d1df-31', '0_AAR-POS-d2a4d1df-21', '0_JAG-UCB-cedd89ab-2', '0_AAR-POS-0daf6b7e-34', '0_AAR-POS-d2a4d1df-26', '0_AAR-POS-0daf6b7e-46', '1_14', '1_18', '1_74', '1_96', '1_132', '0_AAR-POS-d2a4d1df-6', '0_AAR-POS-0daf6b

In [26]:
len(mol_ids)

576

In [27]:
len(set(mol_ids))

554

In [21]:
ids_found = []
for cluster_id, cluster in clusters.items():
    print(f"Cluster {cluster_id}")
    descendents = get_descendents(cluster)
    print(f"Children: {len(descendents)}")
    mols = []
    for desc in descendents:
        mol = desc.repr
        mol.SetTitle(desc.children[0])
        mols.append(mol)
    l.plot_ligands_with_mcs(filename=f"cluster_{cluster_id}.png", mols=mols, mcs_mol=cluster.repr, reference="largest")
    ids_found.extend([desc.children[0] for desc in descendents])

Cluster 0_MAK-UNK-6435e6c2-8
Children: 1
1 molecules to plot
['MAK-UNK-6435e6c2-8']
[11]
Generating a figure with 1 rows and 1 columns
[]
Cluster 0_AAR-POS-0daf6b7e-36
Children: 1
1 molecules to plot
['AAR-POS-0daf6b7e-36']
[15]
Generating a figure with 1 rows and 1 columns
[]
Cluster 0_AAR-POS-0daf6b7e-28
Children: 1
1 molecules to plot
['AAR-POS-0daf6b7e-28']
[15]
Generating a figure with 1 rows and 1 columns
[]
Cluster 0_AAR-POS-d2a4d1df-9
Children: 1
1 molecules to plot
['AAR-POS-d2a4d1df-9']
[15]
Generating a figure with 1 rows and 1 columns
[]
Cluster 0_AAR-POS-0daf6b7e-5
Children: 1
1 molecules to plot
['AAR-POS-0daf6b7e-5']
[18]
Generating a figure with 1 rows and 1 columns
[]
Cluster 0_AAR-POS-f650c5f2-2
Children: 1
1 molecules to plot
['AAR-POS-f650c5f2-2']
[7]
Generating a figure with 1 rows and 1 columns
[]
Cluster 0_VIR-GIT-7b3d3065-2
Children: 1
1 molecules to plot
['VIR-GIT-7b3d3065-2']
[17]
Generating a figure with 1 rows and 1 columns
[]
Cluster 0_AAR-POS-d2a4d1df-8
Ch

1 4
1 5
1 6
2 1
2 2
2 3
2 4
2 5
2 6
3 1
3 2
3 3
3 4
3 5
3 6
4 1
4 2
4 3
4 4
4 5
4 6
5 1
5 2
5 3
5 4
5 5
5 6
6 1
6 2
6 3
Cluster 12_6
Children: 11
11 molecules to plot
['VLA-UCB-29506327-1', 'VLA-UCB-34f3ed0c-11', 'BEN-BAS-c2bc0d80-7', 'VLA-UCB-29506327-1', 'VLA-UCB-29506327-1', 'VLA-UCB-29506327-1', 'VLA-UCB-34f3ed0c-11', 'MAT-POS-8293a91a-8', 'ALP-POS-ce760d3f-8', 'ALP-POS-ce760d3f-8', 'LON-WEI-8f408cad-1']
[27 27 28 27 27 27 27 26 24 24 16]
Generating a figure with 3 rows and 4 columns
['VLA-UCB-29506327-1', 'VLA-UCB-34f3ed0c-11', 'VLA-UCB-29506327-1', 'VLA-UCB-29506327-1', 'VLA-UCB-29506327-1', 'VLA-UCB-34f3ed0c-11', 'MAT-POS-8293a91a-8', 'ALP-POS-ce760d3f-8', 'ALP-POS-ce760d3f-8', 'LON-WEI-8f408cad-1']
1 2
1 3
1 4
2 1
2 2
2 3
2 4
3 1
3 2
3 3
Cluster 3_55
Children: 2
2 molecules to plot
['AAR-POS-0daf6b7e-41', 'MAT-POS-86c60949-2']
[17 28]
Generating a figure with 1 rows and 2 columns
['AAR-POS-0daf6b7e-41']
1 2
Cluster 12_5
Children: 8
8 molecules to plot
['MAT-POS-590ac91e-18', 'M

1 5
1 6
1 7
1 8


1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
4 17
4 18
4 19
4 20
4 21
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
5 20
5 21
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
6 16
6 17
6 18
6 19
6 20
6 21
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
7 13
7 14
7 15
7 16
7 17
7 18
7 19
7 20
7 21
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12
8 13
8 14
8 15
8 16
8 17
8 18
8 19
8 20
8 21
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
9 12
9 13
9 14
9 15
9 16
9 17
9 18
9 19
9 20
9 21
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
10 12
10 13
10 14
10 15
10 16
10 17
10 18
10 19
10 20
10 21
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10


In [32]:
total_mols_found = []
for cluster_id, cluster in clusters.items():
    total_mols_found.extend([desc.children[0] for desc in get_descendents(cluster)])

In [33]:
len(total_mols_found)

1176

In [36]:
len(set(total_mols_found))

554

In [18]:
set(ids_found)

{'AAR-POS-0daf6b7e-1',
 'AAR-POS-0daf6b7e-10',
 'AAR-POS-0daf6b7e-14',
 'AAR-POS-0daf6b7e-15',
 'AAR-POS-0daf6b7e-16',
 'AAR-POS-0daf6b7e-18',
 'AAR-POS-0daf6b7e-2',
 'AAR-POS-0daf6b7e-20',
 'AAR-POS-0daf6b7e-21',
 'AAR-POS-0daf6b7e-22',
 'AAR-POS-0daf6b7e-23',
 'AAR-POS-0daf6b7e-24',
 'AAR-POS-0daf6b7e-25',
 'AAR-POS-0daf6b7e-27',
 'AAR-POS-0daf6b7e-28',
 'AAR-POS-0daf6b7e-29',
 'AAR-POS-0daf6b7e-30',
 'AAR-POS-0daf6b7e-32',
 'AAR-POS-0daf6b7e-33',
 'AAR-POS-0daf6b7e-34',
 'AAR-POS-0daf6b7e-35',
 'AAR-POS-0daf6b7e-36',
 'AAR-POS-0daf6b7e-37',
 'AAR-POS-0daf6b7e-38',
 'AAR-POS-0daf6b7e-39',
 'AAR-POS-0daf6b7e-4',
 'AAR-POS-0daf6b7e-40',
 'AAR-POS-0daf6b7e-41',
 'AAR-POS-0daf6b7e-42',
 'AAR-POS-0daf6b7e-43',
 'AAR-POS-0daf6b7e-44',
 'AAR-POS-0daf6b7e-45',
 'AAR-POS-0daf6b7e-46',
 'AAR-POS-0daf6b7e-5',
 'AAR-POS-0daf6b7e-6',
 'AAR-POS-0daf6b7e-7',
 'AAR-POS-0daf6b7e-8',
 'AAR-POS-5507155c-1',
 'AAR-POS-5507155c-2',
 'AAR-POS-d2a4d1df-1',
 'AAR-POS-d2a4d1df-10',
 'AAR-POS-d2a4d1df-11',
 '

In [19]:
set(mol_ids) - set(ids_found)

set()

# Why are there still duplicates?

In [38]:
from collections import Counter

In [40]:
count_dict = Counter(total_mols_found)

In [54]:
bigger_than_one_count_dict = {k:v for k,v in count_dict.items()
                              if v > 1}

In [55]:
len(bigger_than_one_count_dict)

229

In [56]:
_id, _count = bigger_than_one_count_dict.popitem()

In [67]:
_count

2

## which clusters is this one in?

In [80]:
_id

'TRY-UNI-714a760b-20'

In [81]:
desc_id_dict = {cluster_id: [desc.children[0] for desc in get_descendents(cluster)]
                             for cluster_id, cluster in clusters.items()}

In [91]:
owners = [cluster_id for cluster_id, ids in desc_id_dict.items() if _id in ids]

In [92]:
owners

['18_2']

## get descendents 

In [95]:
cluster0 = clusters[owners[0]]
previous_owners = [cluster.cluster_id for cluster in cluster0.children
                   if _id in [desc.children[0] for desc in get_descendents(cluster)]]

In [96]:
previous_owners

['5_29']

In [97]:
cluster0.children

[ClusterCenter(cluster_id='17_3', children=[ClusterCenter(cluster_id='15_2', children=[ClusterCenter(cluster_id='14_2', children=[ClusterCenter(cluster_id='12_2', children=[ClusterCenter(cluster_id='11_2', children=[ClusterCenter(cluster_id='10_5', children=[ClusterCenter(cluster_id='9_5', children=[ClusterCenter(cluster_id='8_5', children=[ClusterCenter(cluster_id='7_6', children=[ClusterCenter(cluster_id='6_9', children=[ClusterCenter(cluster_id='5_13', children=[ClusterCenter(cluster_id='4_20', children=[ClusterCenter(cluster_id='3_31', children=[ClusterCenter(cluster_id='2_48', children=[ClusterCenter(cluster_id='1_81', children=[ClusterCenter(cluster_id='0_EDG-MED-0da5ad92-18', children=['EDG-MED-0da5ad92-18'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad197dc6c0> >, height=0), ClusterCenter(cluster_id='0_JAN-GHE-83b26c96-22', children=['JAN-GHE-83b26c96-22'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad197f61f0> >, h

In [102]:
cluster1 = cluster0.children[1]
previous_owners = [cluster.cluster_id for cluster in cluster1.children
                   if _id in [desc.children[0] for desc in get_descendents(cluster)]]
print(cluster1.children)
print(previous_owners)

[ClusterCenter(cluster_id='3_35', children=[ClusterCenter(cluster_id='2_55', children=[ClusterCenter(cluster_id='1_104', children=[ClusterCenter(cluster_id='0_TRY-UNI-714a760b-20', children=['TRY-UNI-714a760b-20'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad197df7e0> >, height=0), ClusterCenter(cluster_id='0_TRY-UNI-714a760b-22', children=['TRY-UNI-714a760b-22'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad196148d0> >, height=0)], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad19527bd0> >, height=1), ClusterCenter(cluster_id='1_138', children=[ClusterCenter(cluster_id='0_TRY-UNI-714a760b-19', children=['TRY-UNI-714a760b-19'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad196166a0> >, height=0), ClusterCenter(cluster_id='0_TRY-UNI-714a760b-20', children=['TRY-UNI-714a760b-20'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad197df7e0> >,

In [106]:
cluster2 = cluster1.children[0]
previous_owners = [cluster.cluster_id for cluster in cluster2.children
                   if _id in [desc.children[0] for desc in get_descendents(cluster)]]
print(cluster2.children)
print(previous_owners)

[ClusterCenter(cluster_id='2_55', children=[ClusterCenter(cluster_id='1_104', children=[ClusterCenter(cluster_id='0_TRY-UNI-714a760b-20', children=['TRY-UNI-714a760b-20'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad197df7e0> >, height=0), ClusterCenter(cluster_id='0_TRY-UNI-714a760b-22', children=['TRY-UNI-714a760b-22'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad196148d0> >, height=0)], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad19527bd0> >, height=1), ClusterCenter(cluster_id='1_138', children=[ClusterCenter(cluster_id='0_TRY-UNI-714a760b-19', children=['TRY-UNI-714a760b-19'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad196166a0> >, height=0), ClusterCenter(cluster_id='0_TRY-UNI-714a760b-20', children=['TRY-UNI-714a760b-20'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad197df7e0> >, height=0)], repr=<oechem.OEMol; proxy of <

In [107]:
cluster3 = cluster2.children[0]
previous_owners = [cluster.cluster_id for cluster in cluster3.children
                   if _id in [desc.children[0] for desc in get_descendents(cluster)]]
print(cluster3.children)
print(previous_owners)

[ClusterCenter(cluster_id='1_104', children=[ClusterCenter(cluster_id='0_TRY-UNI-714a760b-20', children=['TRY-UNI-714a760b-20'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad197df7e0> >, height=0), ClusterCenter(cluster_id='0_TRY-UNI-714a760b-22', children=['TRY-UNI-714a760b-22'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad196148d0> >, height=0)], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad19527bd0> >, height=1), ClusterCenter(cluster_id='1_138', children=[ClusterCenter(cluster_id='0_TRY-UNI-714a760b-19', children=['TRY-UNI-714a760b-19'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad196166a0> >, height=0), ClusterCenter(cluster_id='0_TRY-UNI-714a760b-20', children=['TRY-UNI-714a760b-20'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad197df7e0> >, height=0)], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7

In [110]:
cluster3.children[0].children[0].children[0]

'TRY-UNI-714a760b-20'

In [112]:
cluster3.children[1].children[1].children[0]

'TRY-UNI-714a760b-20'

In [116]:
og_count_dict = Counter(mol_ids)
og_count_dict['TRY-UNI-714a760b-20']

1

In [118]:
cluster3.children[0]

ClusterCenter(cluster_id='1_104', children=[ClusterCenter(cluster_id='0_TRY-UNI-714a760b-20', children=['TRY-UNI-714a760b-20'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad197df7e0> >, height=0), ClusterCenter(cluster_id='0_TRY-UNI-714a760b-22', children=['TRY-UNI-714a760b-22'], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad196148d0> >, height=0)], repr=<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fad19527bd0> >, height=1)

In [119]:
# list of og pairs

In [120]:
pairs = [(0, 100), (2, 427), (3, 135), (4, 19), (5, 33), (8, 367), (10, 110), (12, 358), (14, 120), (15, 494), (17, 575), (18, 280), (20, 512), (21, 22), (23, 240), (24, 27), (25, 515), (26, 521), (28, 529), (30, 201), (31, 121), (32, 281), (35, 123), (37, 260), (38, 56), (39, 278), (42, 524), (43, 46), (44, 528), (45, 84), (47, 229), (49, 534), (51, 149), (53, 179), (59, 81), (61, 377), (62, 385), (63, 174), (64, 371), (65, 549), (72, 104), (75, 255), (77, 88), (78, 382), (82, 410), (89, 458), (98, 533), (101, 376), (102, 193), (105, 522), (106, 164), (113, 392), (114, 547), (115, 449), (116, 345), (124, 446), (125, 348), (126, 321), (127, 402), (128, 39), (130, 235), (133, 347), (136, 542), (138, 567), (139, 231), (144, 18), (147, 571), (150, 157), (153, 434), (159, 535), (161, 217), (166, 543), (169, 113), (171, 180), (172, 384), (175, 177), (176, 296), (182, 190), (187, 511), (189, 466), (197, 419), (198, 133), (207, 527), (210, 546), (213, 304), (223, 305), (225, 237), (228, 351), (233, 573), (239, 303), (241, 259), (249, 496), (253, 562), (257, 372), (263, 413), (266, 431), (267, 399), (269, 313), (273, 381), (274, 415), (283, 460), (284, 532), (288, 412), (291, 505), (292, 435), (293, 423), (297, 561), (300, 342), (302, 363), (308, 482), (309, 545), (310, 501), (314, 484), (316, 456), (326, 574), (339, 273), (341, 368), (344, 380), (360, 187), (364, 375), (370, 520), (389, 25), (393, 498), (394, 480), (404, 554), (409, 176), (418, 572), (424, 455), (428, 452), (433, 439), (437, 166), (441, 472), (444, 326), (447, 468), (451, 552), (462, 565), (467, 297), (475, 550), (488, 292), (506, 293)]